In [12]:


from plants_sm.unsupervised.tsne import SklearnTSNE
from plants_sm.unsupervised.pca import SklearnPCA
import numpy as np
from sklearn.preprocessing import StandardScaler
from plants_sm.data_exploration.boxplot import Boxplot
from plants_sm.data_structures.dataset import PandasDataset

dataset2 = PandasDataset(labels_field=["class"],
                         representation_field="sequence_test",
                         instances_ids_field="gene", features_fields=slice(8, -1))
dataset2.from_csv("../data/train_set.tsv")

scaler = StandardScaler()
data = scaler.fit_transform(dataset2.features)

from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score, precision_score, roc_auc_score


def train_and_test(model, train_data, train_labels, test_data, test_labels):
    model.fit(train_data, train_labels)
    predictions = model.predict(test_data)

    cnf_matrix = confusion_matrix(test_labels, predictions)

    print(cnf_matrix)

    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    print('FP: ' + str(FP))
    FN = FN.astype(float)
    print('Fn: ' + str(FN))
    TP = TP.astype(float)
    print('FN: ' + str(FN))
    TN = TN.astype(float)
    print('TN: ' + str(TN))
    # false positive rate
    FPR = FP / (FP + TN)
    print('FPR: ' + str(FPR))
    # False negative rate
    FNR = FN / (TP + FN)
    print('FNR: ' + str(FNR))

    print(classification_report(test_labels, predictions))
    print(f1_score(test_labels, predictions, average="binary"))
    print(recall_score(test_labels, predictions, average="binary"))
    print(precision_score(test_labels, predictions, average="binary"))
    print(roc_auc_score(test_labels, predictions))
    print()
    return model


dataset2 = PandasDataset(labels_field=["class"],
                         representation_field="sequence_test",
                         instances_ids_field="gene", features_fields=slice(8, -1))
dataset2.from_csv("../data/train_set.tsv")
dataset_test = PandasDataset(labels_field=["class"],
                             representation_field="sequence_test",
                             instances_ids_field="gene", features_fields=slice(8, -1))
dataset_test.from_csv("../data/test_set.tsv")
scaler = StandardScaler()
data = scaler.fit_transform(dataset2.features)

data_test = scaler.transform(dataset_test.features)
# Model evaluation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

train_labels = np.select([dataset2.labels[:, 0] == "SM", dataset2.labels[:, 0] == "GM"], [1, 0], dataset2.labels[:, 0])

test_labels = np.select([dataset_test.labels[:, 0] == "SM", dataset_test.labels[:, 0] == "GM"], [1, 0],
                        dataset_test.labels[:, 0])

param_space = {"bootstrap": [True, False],
               "max_depth": [6, 8, 10, 12, 14],
               "max_features": ['auto', 'sqrt', 'log2'],
               "min_samples_leaf": [2, 3, 4],
               "min_samples_split": [2, 3, 4, 5],
               "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
               "class_weight": [{0: 1, 1: 8}]
               }

forest_clf = RandomForestClassifier(class_weight={0: 1, 1: 8})

forest_rand_search = RandomizedSearchCV(forest_clf, param_space, n_iter=100,
                                        scoring="f1", verbose=True, cv=7,
                                        n_jobs=8, random_state=42)
model = train_and_test(forest_rand_search, data, train_labels.astype('int'), data_test, test_labels.astype('int'))

#model = KNeighborsClassifier(n_neighbors=5, weights="distance")
#model = SVC(C=2,kernel="sigmoid", class_weight={0:1, 1: 8})
#model = train_and_test(forest_rand_search, data, train_labels.astype('int'), data_test, test_labels.astype('int'))

/home/joao/Desktop/PlantsSM/src/plants_sm/io/csv.py:59: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self.buffer, **self.kwargs)
/home/joao/Desktop/PlantsSM/src/plants_sm/io/csv.py:59: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self.buffer, **self.kwargs)


Fitting 7 folds for each of 100 candidates, totalling 700 fits
[[153   9]
 [ 13   7]]
FP: [13.  9.]
Fn: [ 9. 13.]
FN: [ 9. 13.]
TN: [  7. 153.]
FPR: [0.65       0.05555556]
FNR: [0.05555556 0.65      ]
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       162
           1       0.44      0.35      0.39        20

    accuracy                           0.88       182
   macro avg       0.68      0.65      0.66       182
weighted avg       0.87      0.88      0.87       182

0.38888888888888884
0.35
0.4375
0.6472222222222223



In [9]:
import sklearn

sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

In [11]:
from collections import Counter

dataset_test = PandasDataset(labels_field=["class"],
                             representation_field="sequence_test",
                             instances_ids_field="gene", features_fields=slice(8, -1))
dataset_test.from_csv("../data/test_set.tsv")

dataset_all_predicted = PandasDataset(labels_field=["union class", "Aracyc class"],
                         representation_field="sequence_test",
                         instances_ids_field="gene", features_fields=slice(8, None))

dataset_all_predicted.from_csv("../data/dados_propythia.csv")


dataframe = dataset_all_predicted.dataframe
df = dataframe[dataframe["gene"].isin(dataset_test.identifiers)]

Counter(df["union class"])


/home/joao/anaconda3/envs/PlantsSM/lib/python3.7/site-packages/dask/core.py:119: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


Counter({'GM-GM': 117, 'GM-SM': 45, 'SM-SM': 16, 'SM-GM': 4})